### 透過臉書facebook圖形API測試工具來找貼文的內容，爬取91app的臉書訊息

In [2]:
import requests
import json
import pandas as pd
#import csv
#from wordcloud import WordCloud
import matplotlib.pyplot as plt
import jieba
import numpy as np

In [4]:
#讀取91app的網頁，分成三個部份
#臉書圖形api網址及版本
head="https://graph.facebook.com/v2.12/"
#我們想要爬取的id，貼文，取前五篇(limit=5)，時間區間設在12/1-12/31(since,until要同時使用)
whoandtime="1498922743698823/posts?limit=5&since=2017-12-01&until=2017-12-31&access_token="
#每兩小時會改變一次，只要修改這裡即可
token="EAACEdEose0cBAEQ3Tg9orjcTGDZATTAXJRJrA6OkjoWMntNizWFXyEbkoZCXWaFdFK9cm6C058sTTbu55ayBNXZBVAc1BbDetDJm3KrpWprpVZCTrd3AV0h53kr8n8CbT8Y0dRyajXNE4jsvcW3889fNFjVKRNZA0bycZBPox0axYZAVHH6cIBoQVYhXT3xRw4ZD"
#網頁
url = head+whoandtime+token

In [5]:
response = requests.get(url)
html = json.loads(response.text) 
html #複製到記事本檢查可發現只呈現了一頁，還有下頁未完 (next 在paging裡面)

{'data': [{'created_time': '2017-12-29T13:00:00+0000',
   'id': '1498922743698823_2027662857491473',
   'message': '✨2018年依舊會是創作者大放異彩的一年!!!'},
  {'created_time': '2017-12-29T04:32:29+0000',
   'id': '1498922743698823_2027524624171963',
   'message': '✨影片已成為2018數位行銷主戰場。'},
  {'created_time': '2017-12-28T14:04:30+0000',
   'id': '1498922743698823_2027216010869491',
   'message': '【91嚴選!】aPure，人間的臭腳終結者。\n看看一家從解決老闆自身腳臭問題起家，\n一步一腳印，走過13年市場變化，\n最終成為台灣臭腳終結者的故事。'},
  {'created_time': '2017-12-27T04:05:42+0000',
   'id': '1498922743698823_2026581374266288',
   'message': '✨2017年大家都在Google什麼？',
   'story': "91APP 品牌新零售 shared 廣告裁判's video."},
  {'created_time': '2017-12-26T13:31:01+0000',
   'id': '1498922743698823_2026283177629441',
   'message': '✨連攝影師也要加入AI失業大軍的行列了是否？'}],
 'paging': {'cursors': {'after': 'Q2c4U1pXNTBYM0YxWlhKNVgzTjBiM0o1WDJsa0R5VXhORGs0T1RJeU56UXpOams0T0RJek9pMDFOelF3T1RnMU1UVTFPREl3TVRFNE5UVXlEd3hoY0dsZAmMzUnZAjbmxmYVdRUElURTBPVGc1TWpJM05ETTJPVGc0TWpOZAk1qQXlOakk0TXpFM056WXl

### 爬取很多頁的貼文 

In [9]:
#讀取下一頁的網址，就在next下面那行即是下一頁的網址
nextflg= html['paging']['next'] 
#paging下包含cursors和next，而cursors下包含after及before
#抓出下一個游標(after那部份的內容)
after  = html['paging']['cursors']['after']
#把上面的html存成data.frame的格式
s1 = pd.DataFrame.from_dict(html['data'])

#將第一頁的貼文存成文字檔
#定義檔名為1的txt檔，存在此程式下的資料夾
filename = "1"+".txt"
s1.to_csv(filename, sep='\t', index=False)

## 爬到最後一頁時，可能不會再出現next，則該如何處理?使用try except或是if else來避免這個問題 

In [13]:
count=1
while nextflg != None:  
  count=count+1
  attrs  = "1498922743698823/posts?pretty=0&since=2017-12-01&until=2017-12-31&limit=5&after="+after+"&access_token="  
  url = head+attrs+token
  response = requests.get(url)
  html = json.loads(response.text) 
  s1 = pd.DataFrame.from_dict(html['data'])
  filename = str(count)+".txt"
  s1.to_csv(filename, sep='\t', index=False)
  after  = html['paging']['cursors']['after']
  try:
      nextflg = html['paging']['next']
  except:
      nextflg = None